In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
#from imblearn.over_sampling import *
import xgboost as xgb
from sklearn.inspection import permutation_importance
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import accuracy_score

In [2]:
datos = pd.read_csv("../2_DataPreparation/DataPrepared_180.csv")

In [3]:
datos.head()

,id,Window,glucose_value,difference_glucose_normal,is_diabetes_diabetes,is_diabetes_normal,is_diabetes_prediabetes,hgb_value,hgb_class_High,hgb_class_Low,...,Drug_DailyDosage_DurationDose_valsartan-640-14,WindowCombinationDrugs_metoprolol-metoprolol-metoprolol-100-50-100,WindowCombinationDrugs_metoprolol-metoprolol-valsartan-50-100-640,WindowCombinationDrugs_pitavastatin-2,Drug_DailyDosage_DurationDose_pitavastatin-2-3,WindowCombinationDrugs_atorvastatin-atorvastatin-metformin-20-20-1000,WindowCombinationDrugs_atorvastatin-losartan-losartan-losartan-10-100-100-100,WindowCombinationDrugs_atorvastatin-atorvastatin-losartan-losartan-losartan-10-10-100-100-100,WindowCombinationDrugs_atorvastatin-atorvastatin-losartan-losartan-10-10-100-100,Stage_Progress
0,0,1080,5.780000,1.920000,0,1,0,12.658177,0,1,...,0,0,0,0,0,0,0,0,0,1
1,0,1260,5.780000,1.920000,0,1,0,13.645556,0,1,...,0,0,0,0,0,0,0,0,0,1
2,0,180,6.632486,1.067514,0,1,0,13.460939,0,1,...,0,0,0,0,0,0,0,0,0,1
3,0,360,7.121271,0.578729,0,1,0,13.162099,0,1,...,0,0,0,0,0,0,0,0,0,1
4,0,540,6.904586,0.795414,0,1,0,13.261657,0,1,...,0,0,0,0,0,0,0,0,0,1


In [4]:
datosResponse = datos["Stage_Progress"].to_numpy()

In [5]:
#datos = datos[["CentroEntrega","RegionEntrega","CantidadDespuesCritica","VolumenHLOrdenado","SubCanal"]]
datos = datos.loc[:, datos.columns != 'Stage_Progress']
datos = datos.loc[:, datos.columns != 'id']
datos = datos.loc[:, datos.columns != 'Window']
datos = datos.loc[:, datos.columns != 'start_day']
datos = datos.loc[:, datos.columns != 'end_day']

In [6]:
for i in datos.columns:
    print(i)

glucose_value
difference_glucose_normal
is_diabetes_diabetes
is_diabetes_normal
is_diabetes_prediabetes
hgb_value
hgb_class_High
hgb_class_Low
hgb_class_Normal
sbp_value
dbp_value
sbp_diff_normal
dbp_diff_normal
bp_class_Elevated
bp_class_Hypertension1
bp_class_Hypertension2
bp_class_HypertensiveCrisis
bp_class_Normal
bp_is_controlled_0
bp_is_controlled_1
ldl_value
diff_Ldl_normal
LDL_class_BorderlineHigh
LDL_class_High
LDL_class_NearNormal
LDL_class_Normal
LDL_class_VeryHigh
creatinine_value
GFR
gfr_evolution
gfr_class_G1
gfr_class_G2
gfr_class_G3a
gfr_class_G3b
gfr_class_G4
gfr_class_G5
age
race_Asian
race_Black
race_Hispanic
race_Unknown
race_White
gender_Female
gender_Male
durationDose
WindowNumberDrugs
TotalDailyDosage
drug_atorvastatin
drug_losartan
drug_metformin
WindowCombinationDrugs_atorvastatin-atorvastatin-losartan-metformin-metformin-10-10-100-1000-1000
Drug_DailyDosage_DurationDose_atorvastatin-10-90
Drug_DailyDosage_DurationDose_losartan-100-270
Drug_DailyDosage_Duration

In [7]:
X_train_full, X_test, y_train_full, y_test = train_test_split(datos, datosResponse)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)
scaler = StandardScaler()


X_train = pd.DataFrame(scaler.fit_transform(X_train.values), columns=X_train.columns, index=X_train.index)


#X_train[X_train.columns] = scaler.fit_transform(X_train)
####AQUI DEBE DE IR SMOTE, EN X_train, ACTUALIZAR TAMBIEN EL NPS TARGET CON DUMMIES DE TODOS
##Hacemos oversampling SMOTE
#sm = BorderlineSMOTE(random_state = 99, n_jobs= -1, kind='borderline-1')
#sm = SMOTE(random_state = 99, n_jobs= -1)
#X_train, y_train = sm.fit_resample(X_train, y_train)

X_valid = pd.DataFrame(scaler.transform(X_valid.values), columns=X_valid.columns, index=X_valid.index)


#X_valid[X_valid.columns] = scaler.transform(X_valid)
#X_test[X_test.columns] = scaler.transform(X_test)


X_test = pd.DataFrame(scaler.transform(X_test.values), columns=X_test.columns, index=X_test.index)



In [8]:
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42, eval_metric="auc", n_estimators=100,
                             max_depth=18, booster = "gbtree", reg_lambda = 3, gamma = 0, eta = 0.3, tree_method = "exact",min_child_weight=1,
                                     grow_policy = "lossguide", scale_pos_weight = 5)

In [9]:
#binary:logistic multi:softmax
xgb_model.fit(X_train, y_train, early_stopping_rounds=10, eval_set=[(X_test, y_test)], verbose=True)

[0]	validation_0-auc:0.75833


c:\users\daniel\appdata\local\programs\python\python37\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[1]	validation_0-auc:0.81552
[2]	validation_0-auc:0.84133
[3]	validation_0-auc:0.85708
[4]	validation_0-auc:0.86501
[5]	validation_0-auc:0.88066
[6]	validation_0-auc:0.89538
[7]	validation_0-auc:0.90391
[8]	validation_0-auc:0.90929
[9]	validation_0-auc:0.91614
[10]	validation_0-auc:0.91671
[11]	validation_0-auc:0.91896
[12]	validation_0-auc:0.92151
[13]	validation_0-auc:0.92315
[14]	validation_0-auc:0.92502
[15]	validation_0-auc:0.93197
[16]	validation_0-auc:0.93442
[17]	validation_0-auc:0.93495
[18]	validation_0-auc:0.93654
[19]	validation_0-auc:0.93684
[20]	validation_0-auc:0.93864
[21]	validation_0-auc:0.93871
[22]	validation_0-auc:0.93815
[23]	validation_0-auc:0.93926
[24]	validation_0-auc:0.93982
[25]	validation_0-auc:0.94025
[26]	validation_0-auc:0.94128
[27]	validation_0-auc:0.94157
[28]	validation_0-auc:0.94325
[29]	validation_0-auc:0.94469
[30]	validation_0-auc:0.94422
[31]	validation_0-auc:0.94460
[32]	validation_0-auc:0.94528
[33]	validation_0-auc:0.94654
[34]	validation_0-a

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eta=0.3,
              eval_metric='auc', gamma=0, gpu_id=-1, grow_policy='lossguide',
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=18,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=12, num_parallel_tree=1, random_state=42,
              reg_alpha=0, reg_lambda=3, scale_pos_weight=5, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [10]:
from sklearn.metrics import classification_report

y_predOrig = xgb_model.predict(X_test)
#y_pred_bool = np.argmax(y_pred, axis=1)
#y_pred_bool = pd.get_dummies(y_pred_bool)

print(classification_report(y_test, y_predOrig))

              precision    recall  f1-score   support

           0       0.89      0.93      0.91       323
           1       0.86      0.80      0.83       187

    accuracy                           0.88       510
   macro avg       0.88      0.86      0.87       510
weighted avg       0.88      0.88      0.88       510



In [11]:
from sklearn.metrics import classification_report

y_predOrig = xgb_model.predict(X_valid)
#y_pred_bool = np.argmax(y_pred, axis=1)
#y_pred_bool = pd.get_dummies(y_pred_bool)

print(classification_report(y_valid, y_predOrig))

              precision    recall  f1-score   support

           0       0.91      0.92      0.92       269
           1       0.81      0.78      0.79       114

    accuracy                           0.88       383
   macro avg       0.86      0.85      0.85       383
weighted avg       0.88      0.88      0.88       383



In [12]:
##Cross Validation
kFold = KFold(n_splits = 10)
res = cross_val_score(xgb_model, X_test, y_test, cv=kFold)
print("The Accuracy is: %.2f%% (%.2f%%)" % (res.mean()*100, res.std()*100))

c:\users\daniel\appdata\local\programs\python\python37\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\users\daniel\appdata\local\programs\python\python37\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\users\daniel\appdata\local\pr

The Accuracy is: 79.41% (4.98%)


In [13]:
kFold = KFold(n_splits = 10)
res = cross_val_score(xgb_model, X_train_full, y_train_full, cv=kFold)
print("The Accuracy is: %.2f%% (%.2f%%)" % (res.mean()*100, res.std()*100))

c:\users\daniel\appdata\local\programs\python\python37\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\users\daniel\appdata\local\programs\python\python37\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\users\daniel\appdata\local\pr

The Accuracy is: 91.18% (2.45%)


In [18]:
#####HyperOpt for searching the best hyperparameters
space={'max_depth': hp.quniform("max_depth", 8, 25, 1),'gamma': hp.uniform ('gamma', 0,9),
       'reg_alpha' : hp.quniform('reg_alpha', 0,5,1),'reg_lambda' : hp.uniform('reg_lambda', 0,1),
       'colsample_bytree' : hp.uniform('colsample_bytree', 0.9,1),'min_child_weight' : hp.quniform('min_child_weight', 0, 25, 1),
       'n_estimators': 150,'seed': 0}

In [19]:
kF = KFold(n_splits = 3)

In [47]:
def objective(space):
    clf=xgb.XGBClassifier(objective="binary:logistic",n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],reg_alpha = int(space['reg_alpha']),
                         scale_pos_weight = 7, seed = space['seed'])
    
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    clf.fit(X_train, y_train,eval_set=evaluation, eval_metric="auc",early_stopping_rounds=10,verbose=False)
    pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, pred)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }
            
    

In [26]:
def objective(space, cv = kF):
    clf=xgb.XGBClassifier(objective="binary:logistic",n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],reg_alpha = int(space['reg_alpha']),
                         scale_pos_weight = 7, seed = space['seed'])
    
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    clf.fit(X_train, y_train,eval_set=evaluation, eval_metric="auc",early_stopping_rounds=10,verbose=False)
    #pred = clf.predict(X_test)
    #accuracy = accuracy_score(y_test, pred)
    score = -cross_val_score(clf, X_test, y_test, cv=cv, scoring="accuracy", n_jobs=-1).mean()
    print ("SCORE:", score)
    return {'loss': score, 'status': STATUS_OK }
            
    

In [48]:
trials = Trials()

best_hyperparams = fmin(fn = objective,space = space,algo = tpe.suggest,max_evals = 100,trials = trials)

  0%|                                                                          | 0/100 [00:00<?, ?trial/s, best loss=?]

c:\users\daniel\appdata\local\programs\python\python37\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



SCORE:                                                                                                                 
0.8313725490196079                                                                                                     
  1%|▍                                              | 1/100 [00:01<01:40,  1.01s/trial, best loss: -0.8313725490196079]

c:\users\daniel\appdata\local\programs\python\python37\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



SCORE:                                                                                                                 
0.8470588235294118                                                                                                     
  2%|▉                                              | 2/100 [00:01<01:33,  1.04trial/s, best loss: -0.8470588235294118]

...

SCORE:                                                                                                                 
0.8725490196078431                                                                                                     
 98%|█████████████████████████████████████████████ | 98/100 [01:34<00:02,  1.01s/trial, best loss: -0.9019607843137255]

c:\users\daniel\appdata\local\programs\python\python37\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



SCORE:                                                                                                                 
0.8647058823529412                                                                                                     
 99%|█████████████████████████████████████████████▌| 99/100 [01:35<00:00,  1.05trial/s, best loss: -0.9019607843137255]

c:\users\daniel\appdata\local\programs\python\python37\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



SCORE:                                                                                                                 
0.7901960784313725                                                                                                     
100%|█████████████████████████████████████████████| 100/100 [01:36<00:00,  1.04trial/s, best loss: -0.9019607843137255]


In [49]:
print("The best hyperparameters are : ","\n")
print(best_hyperparams)

The best hyperparameters are :  

{'colsample_bytree': 0.9722922066342968, 'gamma': 0.5818270825470786, 'max_depth': 20.0, 'min_child_weight': 24.0, 'reg_alpha': 0.0, 'reg_lambda': 0.0016247733609914272}


In [69]:
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42, eval_metric="auc", n_estimators=100,max_depth=23,#space['max_depth'],
                              booster = "gbtree", reg_lambda = space['reg_lambda'], gamma = space['gamma'], eta = 0.3, 
                              tree_method = "exact", reg_alpha = space['reg_alpha'],
                              min_child_weight = space['min_child_weight'],colsample_bytree = space['colsample_bytree'],
                             scale_pos_weight = 2)

xgb_model.fit(X_train, y_train, early_stopping_rounds=10, eval_set=[(X_test, y_test)], verbose=True)

[0]	validation_0-auc:0.79154
[1]	validation_0-auc:0.80592
[2]	validation_0-auc:0.84270


c:\users\daniel\appdata\local\programs\python\python37\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[3]	validation_0-auc:0.85908
[4]	validation_0-auc:0.85869
[5]	validation_0-auc:0.86302
[6]	validation_0-auc:0.87055
[7]	validation_0-auc:0.87841
[8]	validation_0-auc:0.88444
[9]	validation_0-auc:0.89388
[10]	validation_0-auc:0.89912
[11]	validation_0-auc:0.90280
[12]	validation_0-auc:0.90679
[13]	validation_0-auc:0.91252
[14]	validation_0-auc:0.91426
[15]	validation_0-auc:0.91576
[16]	validation_0-auc:0.91666
[17]	validation_0-auc:0.92041
[18]	validation_0-auc:0.91997
[19]	validation_0-auc:0.92210
[20]	validation_0-auc:0.92349
[21]	validation_0-auc:0.92194
[22]	validation_0-auc:0.92224
[23]	validation_0-auc:0.92432
[24]	validation_0-auc:0.92451
[25]	validation_0-auc:0.92517
[26]	validation_0-auc:0.92583
[27]	validation_0-auc:0.92788
[28]	validation_0-auc:0.92846
[29]	validation_0-auc:0.92964
[30]	validation_0-auc:0.93068
[31]	validation_0-auc:0.93134
[32]	validation_0-auc:0.93305
[33]	validation_0-auc:0.93373
[34]	validation_0-auc:0.93412
[35]	validation_0-auc:0.93404
[36]	validation_0

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1,
              colsample_bytree=<hyperopt.pyll.base.Apply object at 0x00000216B398FB48>,
              eta=0.3, eval_metric='auc',
              gamma=<hyperopt.pyll.base.Apply object at 0x00000216B398CF48>,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_dep...
              min_child_weight=<hyperopt.pyll.base.Apply object at 0x00000216B398B088>,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=12, num_parallel_tree=1, random_state=42,
              reg_alpha=<hyperopt.pyll.base.Apply object at 0x00000216B398F3C8>,
              reg_lambda=<hyperopt.pyll.base.Apply object at 0x00000216B398F788>,
              scale_pos_weight=2, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [70]:
from sklearn.metrics import classification_report

y_predOrig = xgb_model.predict(X_valid)
#y_pred_bool = np.argmax(y_pred, axis=1)
#y_pred_bool = pd.get_dummies(y_pred_bool)

print(classification_report(y_valid, y_predOrig))

              precision    recall  f1-score   support

           0       0.92      0.95      0.94       269
           1       0.88      0.81      0.84       114

    accuracy                           0.91       383
   macro avg       0.90      0.88      0.89       383
weighted avg       0.91      0.91      0.91       383



In [71]:
from sklearn.metrics import classification_report

y_predOrig = xgb_model.predict(X_test)
#y_pred_bool = np.argmax(y_pred, axis=1)
#y_pred_bool = pd.get_dummies(y_pred_bool)

print(classification_report(y_test, y_predOrig))

              precision    recall  f1-score   support

           0       0.89      0.94      0.92       323
           1       0.88      0.81      0.84       187

    accuracy                           0.89       510
   macro avg       0.89      0.87      0.88       510
weighted avg       0.89      0.89      0.89       510



In [ ]:
from imblearn.ensemble import BalancedBaggingClassifier

In [218]:
brf = BalancedBaggingClassifier(n_estimators = 1200, n_jobs=-1, oob_score = True)
brf.fit(X_train, y_train)

BalancedBaggingClassifier(n_estimators=1200, n_jobs=-1, oob_score=True)

In [219]:
brf.score(X_train,y_train)

0.999128160418483

In [220]:
brf.score(X_test,y_test)

0.8470588235294118

In [221]:
brf.score(X_valid,y_valid)

0.8668407310704961

In [ ]:
##We will select the most important features

In [165]:
perm_importance = permutation_importance(xgb_model, X_train_full, y_train_full)

In [166]:
sorted_idx = perm_importance.importances_mean.argsort()

In [196]:
###We obtain the top 100 features
indexImportancia = sorted_idx[-300:]
print(indexImportancia)

[ 50  62  34  31   1   2   3   5   6   9  10  11  12  15  32  16  19  20
  21  22  23  24  25  26  28  30  17 119  63  65  95  96  97  98  99 100
 101 102 103 104  94 106 108 109 110 111 112 113 114 115 116 117 107  64
  93  89  66  67  69  70  71  72  73  74  75  76  91  77  79  80  81  82
  83  84  85  86  87  88  78 224 219 226 357 358 359 360 361 362 363 364
 365 366 356 367 369 370 371 372 373 374 375 376 377 378 368 379 355 353
 331 332 333 225 335 336 337 338 339 340 354 341 343 344 345 346 347 348
 349 350 351 352 342 380 381 382 410 411 412 413 414 415 416 417 418 419
 409 420 422 423 424 425 426 427 428 429 859 431 421 408 407 406 383 384
 385 386 387 388 389 390 391 392 393 394 395 396 397 398 399 400 401 402
 403 404 405 330 329 334 327 254 255 256 257 258 259 260 261 262 263 264
 265 266 267 268 269 270 271 272 273 274 328 276 253 277 252 250 227 228
 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246
 247 248 249 251 278 275 280 304 305 306 307 308 30

In [197]:
print(datos.columns[indexImportancia])

Index(['WindowCombinationDrugs_atorvastatin-atorvastatin-losartan-metformin-metformin-10-10-100-1000-1000',
       'WindowCombinationDrugs_atorvastatin-atorvastatin-20-20',
       'gfr_class_G4', 'gfr_class_G2', 'difference_glucose_normal',
       'is_diabetes_diabetes', 'is_diabetes_normal', 'hgb_value',
       'hgb_class_High', 'sbp_value',
       ...
       'Drug_DailyDosage_DurationDose_metformin-500-90', 'hgb_class_Normal',
       'drug_rosuvastatin', 'Drug_DailyDosage_DurationDose_losartan-100-90',
       'drug_simvastatin', 'hgb_class_Low', 'gfr_evolution', 'durationDose',
       'drug_atorvastatin', 'gfr_class_G3b'],
      dtype='object', length=300)


In [198]:
X_train_full_filtered, X_test_filtered, y_train_full_filtered, y_test_filtered = train_test_split(datos.iloc[:,indexImportancia], datosResponse)
X_train_filtered, X_valid_filtered, y_train_filtered, y_valid_filtered = train_test_split(X_train_full_filtered, y_train_full_filtered)
scaler = StandardScaler()


X_train_filtered = pd.DataFrame(scaler.fit_transform(X_train_filtered.values), columns=X_train_filtered.columns, index=X_train_filtered.index)


#X_train[X_train.columns] = scaler.fit_transform(X_train)
####AQUI DEBE DE IR SMOTE, EN X_train, ACTUALIZAR TAMBIEN EL NPS TARGET CON DUMMIES DE TODOS
##Hacemos oversampling SMOTE
#sm = BorderlineSMOTE(random_state = 99, n_jobs= -1, kind='borderline-1')
#sm = SMOTE(random_state = 99, n_jobs= -1)
#X_train, y_train = sm.fit_resample(X_train, y_train)

X_valid_filtered = pd.DataFrame(scaler.transform(X_valid_filtered.values), columns=X_valid_filtered.columns, index=X_valid_filtered.index)


#X_valid[X_valid.columns] = scaler.transform(X_valid)
#X_test[X_test.columns] = scaler.transform(X_test)


X_test_filtered = pd.DataFrame(scaler.transform(X_test_filtered.values), columns=X_test_filtered.columns, index=X_test_filtered.index)

In [199]:
xgb_modelFILTERED = xgb.XGBClassifier(objective="binary:logistic", random_state=42, eval_metric="auc", n_estimators=100,max_depth=space['max_depth'],
                              booster = "gbtree", reg_lambda = space['reg_lambda'], gamma = space['gamma'], eta = 0.3, 
                              tree_method = "exact",grow_policy = "lossguide", reg_alpha = space['reg_alpha'], min_child_weight = space['min_child_weight'],colsample_bytree = space['colsample_bytree'],
                             scale_pos_weight = 7)

xgb_modelFILTERED.fit(X_train_filtered, y_train_filtered, early_stopping_rounds=10, eval_set=[(X_test_filtered , y_test_filtered )], verbose=True)

[0]	validation_0-auc:0.69897
[1]	validation_0-auc:0.74542
[2]	validation_0-auc:0.75129
[3]	validation_0-auc:0.77523
[4]	validation_0-auc:0.79891
[5]	validation_0-auc:0.81398
[6]	validation_0-auc:0.81794
[7]	validation_0-auc:0.82712
[8]	validation_0-auc:0.83154


/usr/local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[9]	validation_0-auc:0.82997
[10]	validation_0-auc:0.83118
[11]	validation_0-auc:0.82760
[12]	validation_0-auc:0.82901
[13]	validation_0-auc:0.83198
[14]	validation_0-auc:0.83188
[15]	validation_0-auc:0.83531
[16]	validation_0-auc:0.84247
[17]	validation_0-auc:0.84451
[18]	validation_0-auc:0.84688
[19]	validation_0-auc:0.84756
[20]	validation_0-auc:0.84685
[21]	validation_0-auc:0.84605
[22]	validation_0-auc:0.84698
[23]	validation_0-auc:0.84741
[24]	validation_0-auc:0.84920
[25]	validation_0-auc:0.84864
[26]	validation_0-auc:0.85032
[27]	validation_0-auc:0.85606
[28]	validation_0-auc:0.85548
[29]	validation_0-auc:0.85747
[30]	validation_0-auc:0.85967
[31]	validation_0-auc:0.86383
[32]	validation_0-auc:0.86469
[33]	validation_0-auc:0.86512
[34]	validation_0-auc:0.86716
[35]	validation_0-auc:0.86514
[36]	validation_0-auc:0.86498
[37]	validation_0-auc:0.86461
[38]	validation_0-auc:0.86761
[39]	validation_0-auc:0.86858
[40]	validation_0-auc:0.86843
[41]	validation_0-auc:0.87075
[42]	valida

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1,
              colsample_bytree=<hyperopt.pyll.base.Apply object at 0x1682ba340>,
              eta=0.3, eval_metric='auc',
              gamma=<hyperopt.pyll.base.Apply object at 0x1682b4eb0>, gpu_id=-1,
              grow_policy='lossguide', importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step...
              min_child_weight=<hyperopt.pyll.base.Apply object at 0x1682ba490>,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=8, num_parallel_tree=1, random_state=42,
              reg_alpha=<hyperopt.pyll.base.Apply object at 0x1682ba100>,
              reg_lambda=<hyperopt.pyll.base.Apply object at 0x1682ba220>,
              scale_pos_weight=7, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [200]:

y_predOrig = xgb_modelFILTERED.predict(X_test_filtered)
#y_pred_bool = np.argmax(y_pred, axis=1)
#y_pred_bool = pd.get_dummies(y_pred_bool)

print(classification_report(y_test_filtered, y_predOrig))

              precision    recall  f1-score   support

           0       0.89      0.87      0.88       333
           1       0.76      0.79      0.78       177

    accuracy                           0.84       510
   macro avg       0.82      0.83      0.83       510
weighted avg       0.84      0.84      0.84       510



In [201]:
y_predOrig = xgb_modelFILTERED.predict(X_valid_filtered)
#y_pred_bool = np.argmax(y_pred, axis=1)
#y_pred_bool = pd.get_dummies(y_pred_bool)

print(classification_report(y_valid_filtered, y_predOrig))

              precision    recall  f1-score   support

           0       0.95      0.87      0.91       260
           1       0.76      0.89      0.82       123

    accuracy                           0.88       383
   macro avg       0.85      0.88      0.86       383
weighted avg       0.89      0.88      0.88       383



In [ ]:
from imblearn.ensemble import BalancedBaggingClassifier

In [114]:
brf = BalancedBaggingClassifier(n_estimators = 500, n_jobs=-1, oob_score = True)
brf.fit(X_train, y_train)

BalancedBaggingClassifier(n_estimators=500, n_jobs=-1, oob_score=True)

In [115]:
brf.score(X_train,y_train)

0.999128160418483

In [116]:
brf.score(X_test, y_test)

0.8509803921568627

In [117]:
brf.score(X_valid, y_valid)

0.8642297650130548